# Homework 2
- INFO 4871 _or_ INFO 5871 (indicate)
- Spring 2019
- _Aaron Barge_

## Objective (all students)
The object of this assignment is to implement a basic user-knn recommendation algorithm as a Python class. I am supplying six files to help you do this:

- This file (Jupyter notebook) acts a driver for your class. You will run this notebook _last_ after your implementation is complete to demonstrate that it works.
- The python file `User_KNN.py`, which contains a commented implementation skeleton of the `User_KNN` class. You will finish out this implementation. 
- The python file `test_User_KNN.py`, which contains an incomplete unit test for the `User_KNN` class.
- The Excel file `test_data.xlsx`. You will use this to compute by hand the missing correct values for the tests `test_similarities_hwk2` and `test_predictions_hwk2`. 
- The file `test_ratings.csv`, which contains the same ratings as in the `test_data.xlsx` file.
- The file `ratings_film_3.txt`, which contains movie ratings from the FilmTrust data set. 

Turn in the following in a zip file:

- Homework 2.html file computed from this worksheet after your implementation is complete.
- Homework 2.ipynb (this file) run after your implementation is complete.
- test_User_KNN.py file with the test functions complete and working with your User_KNN implementation
- User_KNN.py file with completed implementation
- test_data.xslx file showing the values needed for the test case computed

## Objective (INFO 5871 only)
In addition to the objectives above, you will implement a second user-knn recommendation algorithm named `User_KNN2`. This one will add the following features to the implementation described above:

- Default prediction. If there are fewer than `min_users` in the prediction neighborhood, the algorithm will return the item mean as the prediction, rather than computing a prediction.
- Shrinkage. The algorithm will implement similarity shrinkage as described in class. All similarities scores will be multiplied by $\frac{|P_u \cap P_v|}{\beta + |P_u \cap P_v|}$, where $\beta$ is supplied in the constructor.
- Generalized labels. The code in `User_KNN.py` assumes that the data frame given to `fit` has columns `'userId'`, `'movieId'` and `'rating'`. `User_KNN2` will accept data frames with arbitrary column names, and the mapping of labels to user, item, and rating is input to the constructor.
- You will write your own unit test for `User_KNN2`, which contains all of the same tests as `test_User_KNN` but with the correct values as computed in a second page of the `test_data.xslx` spreadsheet.

Turn in the following in a zip file:

- Everything from the "all students" objective above.
- User_KNN2.py implementation
- test_User_KNN2.py implementation working with your User_KNN2 implementation

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import logging
from heapq import nlargest
%matplotlib inline


In [3]:
%load_ext autoreload
# %aimport User_KNN
# %autoreload 2

from User_KNN import User_KNN

In [4]:
import unittest
from test_User_KNN import test_User_KNN

%aimport test_User_KNN

ModuleNotFoundError: No module named 'test_User_KNN'

In [10]:
suite = unittest.TestLoader().loadTestsFromTestCase(test_User_KNN.test_User_KNN)
runner = unittest.TextTestRunner(verbosity=2)
runner.run(suite)

NameError: name 'test_User_KNN' is not defined

### Load the FilmTrust ratings
Should be 301 users

In [11]:
ratings = pd.read_csv('ratings_film_3.csv', sep='\s+', names=['userId','movieId','rating'])
len(ratings['userId'].unique())
ratings.head()

,userId,movieId,rating
0,750,205,0.5
1,750,214,0.5
2,750,220,0.5
3,750,256,0.5
4,750,250,0.5


In [12]:
ratings = pd.read_csv('jester_ratings_300.csv', sep=',', header=0, names=['userId','movieId','rating'])
len(ratings['userId'].unique())
# ratings.head()

269

In [13]:
type(ratings)
ratings.index

RangeIndex(start=0, stop=11349, step=1)

In [14]:
ratings

,userId,movieId,rating
0,1,5,0.219
1,1,7,-9.281
2,1,8,-9.281
3,1,13,-6.781
4,1,15,0.875
5,1,16,-9.656
6,1,17,-9.031
7,1,18,-7.469
8,1,19,-8.719
9,1,20,-9.156


### Set up the algorithm
Note this is pretty slow. It could take 10-20 minutes or longer depending on your hardware. It prints out every 10 uesrs, so you can estimate how long it will take to process all 301 of them.

In [23]:
alg = User_KNN(20, sim_threshold=1)
alg.fit(ratings)

In [29]:
alg.predict_for_user(870, [6,1,1], [1,2,3,4,5])

KeyError: 870

In [27]:
alg.predict_for_user(8, [1,3,4,1], [10,10,10,10])

1    1.805525
3    1.805525
4    1.805525
1    1.805525
dtype: float64